In [1]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch

2024-04-24 23:47:13.003078: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 23:47:13.591232: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/undergrad/miniconda3/envs/tf-2.12.0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def detection(final_sequence, logit_shape, gamma) :
    
    input_ids = tokenizer.encode(final_sequence, return_tensors='tf')
    input_ids_list = input_ids.numpy().tolist()[0]
    prev_tk = input_ids[0]
    green_counts = 0
    
    for id in input_ids_list :
        seed = id % logit_shape
        #print(f"seed : {seed[-1]}")
        np.random.seed(seed)
        
        indices = np.arange(logit_shape) #0~51999
        green_list_size = int(logit_shape * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
       #print(f"detection greenList : {green_list}")
        
        if id in green_list :
            green_counts = green_counts + 1
        prev_id = id

    print(f"green list 단어 개수 : {green_counts}")
    print(f"red list 단어 개수 : {len(input_ids_list)-green_counts}")
    green_proportion = green_counts/len(input_ids_list)
    red_proportion = 1-green_proportion
    print(f"green list 단어 비율 : {green_proportion}")
    print(f"red list 단어 비율 : {red_proportion}")
    expected_proportion = 0.5 #green list 를 모를 것이다 -> H0
    std_error = np.sqrt(expected_proportion * (1 - expected_proportion) / len(input_ids))
    z_score = (green_proportion - expected_proportion) / std_error
    p_value = 2 * norm.sf(np.abs(z_score))
    
    print(f"Z-score: {z_score}, P-value: {p_value}")
    if p_value < 0.05 :
        return True
    else :
        return False

#g=0.7, d=10.0 ->red list 단어 비율 : 0.580000000000000
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에는 책 읽는 즐거움이란 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로"
#g=0.5, d=50.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=20.0 -> red list 단어 비율 : 0.53
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 "
#g=0.5, d=10.0 -> red list 단어 비율 : 0.5252525252525253
#sentence = "책 읽기의 즐거움도 함께 느낄 것이다.이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여했다. 이번 전시는 '책과 세상을 잇는 길'이라는 주제 하에 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 작가인 김혜숙씨가 참여한 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는 '책과 세상을 잇는 길'이라는 주제 하에 진행됐고 전시는"
#g=0.5, d=5.0 -> red list 단어 비율 : 0.505050505050505
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책과 나눔'이라는 주제 하에 지난달 27일 개막한 '책과 나눔'전을 통해 처음 선보였으며, 오는 9월까지 계속된다. 특히, 전시 기간 동안 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해서는 '책과 나눔'전을 통해"
#g=0.5, d=1.0 -> red list 단어 비율 : 0.6
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다.이번 전시는 ‘책 읽는 즐거움’을 주제로 한 ‘책 읽는 즐거움’ 전시를 비롯해 ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움’ 전시, ‘책 읽는 즐거움"
#g=1.0, d=0.0 -> red list 단어 비율 : 0.5700000000000001
#sentence = "책 읽기의 즐거움도 함께 느낄 수 있다. 이번 전시는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 진행된다. 전시에서는 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는 주제로 '책 읽는 즐거움'이라는"

#g=0.5
#sentence = "아침 일과 시작하기 전에 미리 준비해두면 좋겠습니다. 오늘은 전국이 대체로 맑겠습니다. 하지만 제주도는 구름이 많이 끼겠고요. 강원 영동 지역은 밤 한때 비가 조금 내리겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다. 서울이 4도, 대구 6도로 출발하겠습니다. 낮 기온은 서울이 13도, 대구와 광주 16도로 오늘과 비슷하겠습니다. 바다의 물결은 동해 먼바다에서 최고 2.5미터까지 높게 일겠습니다. 내일 아침 기온 오늘과 비슷하겠습니다."

#사람
#sentence = "안정된 순방향 전파: 만약 우리가 오프셋 δ를 0으로 초기화하고 스케일 γ를 1로 초기화한다면, 각 출력 활성화는 단위 분산을 갖게 됩니다. 일반적인 네트워크에서는 이는 초기화 과정에서 순방향 전파 중에 분산이 안정적으로 유지됨을 보장합니다. 하지만 잔차 네트워크에서는 분산은 여전히 새로운 변동 요소가 각 레이어에서 입력에 추가되면서 증가해야 합니다. 그러나 잔차 블록마다 분산은 선형적으로 증가할 것입니다. k번째 레이어는 k의 기존"

#사람
#sentence = "이건 사람이 작성한 글입니다. 절대 절대 p-value 가 낮게 나와서도 안되고 z-score가 0 근처여서도 안됩니다. 제발 좀 잘 나와보자 제발제발 할말이 없네 이게 다야"
#detection(sentence, 25600, 0.5)

In [ ]:
# Set the repetition penalty
penalty_coefficient = 1.3

# Function to apply dynamic repetition penalty to logits
def apply_repetition_penalty(logits, token_counts, penalty_coefficient):
    penalty_factors = np.ones(logits.shape[-1])

    for token_id, count in enumerate(token_counts):
        if count > 0:
            penalty_factors[token_id] = penalty_coefficient ** count

    # Apply the calculated penalties to logits
    return logits / penalty_factors

In [ ]:
for line in lines:
    print(f"prompt : {line}")

    input_text = line
    input_ids = tokenizer.encode(input_text, return_tensors='tf')
    #term = 3

    last_index = input_ids.shape[-1]
    nlls = []
    token_counts = np.zeros(tokenizer.vocab_size)
    with mirrored_strategy.scope():
        for _ in tqdm(range(100)):  # 최대 100번 반복
            if input_ids.shape[-1] >= 100:
                break  # input_ids의 길이가 100 이상이면 반복 중단
            
            #print("================================")
            #print(f"문장의 토큰 개수 : {input_ids.shape[-1]}")
            #set 나눠서 green set 로짓을 촉진하기
            gamma = 0.7
            delta = 10.0
            Logits = model(input_ids).logits
            #print(f"original Logits : {Logits}")
            
            #seed 생성
            seed = input_ids[-1] % Logits.shape[-1]
            print(f"seed : {seed[-1]}")
            np.random.seed(seed[-1])

            #random green list 설정
            indices = np.arange(Logits.shape[-1]) #0~51999
            green_list_size = int(Logits.shape[-1] * gamma)  #25600
            green_list = np.random.choice(indices, green_list_size, replace=False)
            print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

            #--여기부터 probability 구하기
            # green logit과 red logit의 값을 담을 리스트 생성
            green_logits = tf.zeros(green_list_size)
            red_logits = tf.zeros(Logits.shape[-1] - green_list_size)

            #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
            Logits_num = np.zeros((0,last_index-1,Logits.shape[-1]))
            Logits_num = Logits.numpy()
            #Logits_num[0, -1, :] = apply_repetition_penalty(Logits_num[0, -1, :], token_counts, penalty_coefficient)
            
            for i in green_list :
                Logits_num[0,last_index-1,i] = Logits_num[0,last_index-1,i]+delta
            Logits = tf.convert_to_tensor(Logits_num, dtype=tf.float32)
            #print(f"updated Logits : {Logits}")
            
            # softmax 계산을 위한 분모
            denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  
            #print(f"denominator : {denominator}") #25600개의 green list

            # Logits 텐서에 softmax 적용하여 확률 계산 (병렬 계산), softmax_tensor = probability
            softmax_tensor = Logits.numpy()
            softmax_tensor[0, last_index-1, :] = np.exp(softmax_tensor[0, last_index-1, :])
            softmax_tensor[0, last_index-1, :] /= denominator[0,last_index-1]
            #print(softmax_tensor.shape)
            #print(f"Probabilities : {softmax_tensor}")
            #print(softmax_tensor[0, last_index-1, add_token_id])

            # 데이터 타입 확인 및 조정
            add_token_id = tf.argmax(softmax_tensor[0][last_index-1])
            #print(f"updated Logits Max value : {add_token_id}")
            
            # Update token counts
            #token_counts[add_token_id] += 1

            
            # Calculate perplexity
            # 참고 https://huggingface.co/docs/transformers/main/ko/perplexity
            selected_probability = softmax_tensor[0, last_index-1, add_token_id]
            nlls.append(np.log(selected_probability))
            
            # 텐서 연결
            add_token_id = tf.reshape(add_token_id, [1,1])
            add_token_id = tf.cast(add_token_id, input_ids.dtype)
            input_ids = tf.concat([input_ids, add_token_id], axis=-1)
            next_token = tokenizer.decode(input_ids.numpy()[0,:])
            print(f"next token : {next_token}")
            
            last_index = input_ids.shape[-1]
            #term = 3
        final_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(final_token)
        print(f"is this sentence written by machine? {detection(final_token, Logits.shape[-1], gamma)}")

        # Calculate perplexity
        perplexity = tf.exp(-tf.reduce_mean(nlls))
        print("Perplexity: ", perplexity.numpy())